In [ ]:
%load_ext autoreload
%autoreload 2
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent.parent))

In [ ]:
%reload_ext dotenv

%dotenv ../../env/.env

import warnings
from pandas import Timedelta
# from optiml.utils import sf
import time
from optiml.utils.sf import logger, sql_to_df, run_sql, conn, session
import pandas as pd
warnings.filterwarnings('ignore')

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime


In [ ]:
# Q1: if Cluster 1 and 3 are active, 2 is idle, and num_clusters is set to 2, does #2 shutoff?
#    1b) are credits saved?
# Q2: if cluster 1 and 2 are active, and 2 is shutoff before 60 seconds of uptime, are we billed
#     at minimum of 60 secs like base warehouse cluster? or charged for usage?
#     - yes, minimum of 60 secs billed

# algos:
# 1) if warehouse is idle, shut down warehouse
# 2) estimate total number of active clusters
#   a) use total number of running queries
#   b) use total pct of provisioned 
# set max_clusters to that number of active clusters
# 3) if multicluster type is STANDARD and warehouse is idle set to ECONOMY
#    if not idle and total active clusters > 1 (or total active clusters = max clusters)
#        then set to ECONOMY

In [ ]:
import time
def monitor():
    show_df = sql_to_df("show warehouses")
    now = pd.Timestamp.now()
    show_df['ts'] = now
    something_running = 'STARTED' in show_df.state.to_list()
    all_show_results = []
    all_show_results.append(show_df)
    
    while something_running:
        time.sleep(1)
        show_df = sql_to_df("show warehouses")
        now = pd.Timestamp.now()
        show_df['ts'] = now
        something_running = 'STARTED' in show_df.state.to_list()
        print(something_running)
        all_show_results.append(show_df)
    return pd.concat(all_show_results)
#     def show():
#         run_sql(f"USE WAREHOUSE {using_wh}", ctx=using_conn)
#         show_df = sql_to_df("show warehouses")

    
#     show()
    print(something_running)

df = monitor()
df

In [ ]:
wh_name = 'test_control_a67fc85c_9de9_400e_8e49_6c93dfcef9db'

# ['test_control_4d79538c_94ab_464d_bb1e_3320be370837',
#  'test_suspend_idle_after_one_minute_11404fa8_0e29_40d7_8eee_9a7dc8414539',
#  'test_suspend_after_one_minute_a3185ebe_291c_4b5e_aeed_cd22efb8457c']

In [ ]:
import pandas as pd
import datetime

current_time = datetime.datetime.now().strftime("%Y_%m%d_%H%M%S")
print(current_time)
# Assuming your DataFrame is called 'df'
file_name = f'{wh_name}_{current_time}.pkl'
print(file_name)
df.to_pickle(file_name)

In [ ]:
# path = 'test_bc9a9691_c770_4eb7_96e9_278aac7db0c6_2023_0609_143717.pkl'
# df = pd.read_pickle(path)

In [ ]:
# wh_name='DEMO'
print(wh_name.upper())
import plotly.express as px
fig = px.line(df[df.name == wh_name.upper()], x='ts', y='started_clusters')
fig.show()
fig = px.line(df[df.name == wh_name.upper()], x='ts', y=['running', 'queued'])
fig.show()
fig = px.line(df[df.name == wh_name.upper()], x='ts', y=['available', 'provisioning', 'quiescing'])
fig.update_yaxes(autorange="reversed")

fig.show()

In [ ]:
wh_status = f"""with qh as (
select 
    *,
    md5(warehouse_name) as warehouse_id,
      warehouse_name || ':' || cluster_number || ':' || query_id as warehouse_query,
    TIMESTAMPADD( millisecond , -execution_time,end_time) as execution_start_time
from table(information_schema.query_history_by_warehouse('{wh_name}', result_limit=>10000))
where execution_status != 'RUNNING'
and warehouse_size is not null
and start_time > '2023-06-10 11:00'
order by start_time desc
),
seconds as (
    select
        dateadd(
            'second',
            '-' || row_number() over (order by seq4() asc),
            dateadd('day', '+1', date_trunc('second', current_timestamp))
        ) as second_start,
        dateadd('second', '+1', second_start) as timestamp
    from table(generator(rowcount => (3600*24 * 100)))
    qualify second_start between (select min(start_time) from qh) and (select max(end_time) from qh)
),
warehouse_cluster_seconds as (
    select * from 
    seconds
    cross join 
    (select distinct md5(warehouse_name) as warehouse_id, warehouse_name, cluster_number from qh) w

)
-- select * from warehouse_seconds;
select
    seconds.timestamp,
    seconds.warehouse_name,
    seconds.warehouse_id,
    seconds.cluster_number,
    count(queries.query_id) as num_queries
from warehouse_cluster_seconds seconds
left join qh as queries
    -- on date_trunc('hour', seconds.timestamp)=queries.query_hour -- NEW: equi-join condition
    on seconds.warehouse_id = queries.warehouse_id
    and seconds.cluster_number = queries.cluster_number
    and seconds.timestamp -- range join condition
      between date_trunc('second', queries.execution_start_time) and date_trunc('second', queries.end_time)
group by 1,2,3,4
"""

wdf = sql_to_df(wh_status)
display(wdf)


events = f"""
select *
from snowflake.account_usage.warehouse_events_history
where lower(warehouse_name) = '{wh_name}'
and timestamp > '2023-06-08 11:00'
"""

whedf = sql_to_df(events)
display(whedf)

In [ ]:
wdf.cluster_number = wdf.cluster_number.map(int)

In [ ]:
import plotly.express as px
fig = px.scatter(whedf.sort_values('warehouse_name'), x="timestamp", y="event_name", color='event_name', hover_data=['cluster_number', 'event_name', 'event_reason'])
fig.show()

In [ ]:
wdf

In [ ]:
import plotly.express as px
wdf['timestamp_est'] = wdf.timestamp + pd.Timedelta(hours=3)

fig = px.area(wdf.sort_values(['timestamp', 'cluster_number']), x='timestamp_est', y=['num_queries'], color = 'cluster_number')
# fig.update_yaxes(autorange="reversed")

fig.show()

In [ ]:
wdf.clus

In [ ]:
[df.ts.min(), df.ts.max()]

In [ ]:
from plotly.subplots import make_subplots

# qdf['warehouse_query'] = qdf.warehouse_name + " : " + qdf.query_id

wdf['timestamp_est'] = wdf.timestamp + pd.Timedelta(hours=3)
whedf['timestamp_est'] = whedf.timestamp + pd.Timedelta(hours=3)


# wh_name='DEMO'
print(wh_name.upper())
import plotly.express as px
# fig = px.line(df[df.name == wh_name.upper()], x='ts', y='started_clusters')
# fig.show()
# fig = px.line(df[df.name == wh_name.upper()], x='ts', y=['running', 'queued'])
# fig.show()
# fig = px.line(df[df.name == wh_name.upper()], x='ts', y=['available', 'provisioning', 'quiescing'])
# fig.update_yaxes(autorange="reversed")

# fig.show()


figures = [
    px.line(df[df.name == wh_name.upper()], x='ts', y='started_clusters'),
    px.line(df[df.name == wh_name.upper()], x='ts', y=['running', 'queued']),
    px.line(df[df.name == wh_name.upper()], x='ts', y=['available', 'provisioning', 'quiescing']),
    px.scatter(whedf.sort_values('warehouse_name'), x="timestamp_est", y="event_name", color='event_name', hover_data=['cluster_number', 'event_name', 'event_reason']),
    px.area(wdf.sort_values(['timestamp_est', 'cluster_number']), x='timestamp_est', y=['num_queries'], color='cluster_number')
    ]

fig = make_subplots(rows=len(figures), cols=1, shared_xaxes=True, vertical_spacing=0.05) 

for i, figure in enumerate(figures):
    for trace in range(len(figure["data"])):
        fig.append_trace(figure["data"][trace], row=i+1, col=1)


fig.update_xaxes(showgrid=True,minor=dict(showgrid=True))
fig.update_yaxes(showgrid=True,minor=dict(showgrid=True))
fig.update_xaxes(autorange=True)

fig.update_xaxes(range=[df.ts.min(), df.ts.max()])
fig.update_layout(
    height=800,
)
fig.update_xaxes(type='date', autorange=True)
fig.update_yaxes(row=3, col=1, autorange='reversed')

# fig.update_layout(
#     xaxis_range=[cqe.era_start.min(), cqe.era_end.max()]  # Specify your desired minimum and maximum range
# )
fig.show()

# next: try just one layer of test query

In [ ]:
df[df.name == wh_name.upper()]